# Подключение библиотек

In [7]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5
import time
import struct
import os

# Подключимся к MetaTrader 5

In [8]:
if not mt5.initialize():
    print("initialize() failed")
    quit()

# Вычисления медианы, верхненего и нижнего пределов

In [9]:
def take_rates():
    utc_from = datetime.now() - timedelta(minutes=15)
    gbprub_rates = mt5.copy_rates_from("GBPUSD", mt5.TIMEFRAME_M15, utc_from, 15)
    usdrub_rates = mt5.copy_rates_from("EURUSD", mt5.TIMEFRAME_M15, utc_from, 15)
    return gbprub_rates, usdrub_rates

In [10]:
def calculation(rate1, rate2, loteuro):
    spread = take_rates()[0]['close'] - take_rates()[1]['close'] * loteuro # Вычисляем спред
    spread = np.array(spread)
    spread = spread[~np.isnan(spread)]
    median = np.median(spread)
    Open_up = list()
    Open_down = list()
    Open_UP =0
    Open_DOWN = 0
    #Вычисляем верхнюю и нижнюю границу
    for i in range(0,len(spread)):
        if spread[i] > median:
            Open_up.append(spread[i])
        if spread[i] < median:
            Open_down.append(spread[i])
    Open_UP =np.median(Open_up)
    Open_DOWN = np.median(Open_down)
    return Open_UP, Open_DOWN, median

# Подготавливаем все запросы
## Нам нужно 8 запросов: 4 для открытия позиции, 4 для закрытия позиции.

In [12]:
symbol_gbp = "GBPUSD"
symbol_euro = "EURUSD"
symbol_gbp_info =mt5.symbol_info(symbol_gbp)
symbol_euro_info =mt5.symbol_info(symbol_euro)
lotgbp = 1.0
loteuro = round(1.0 * symbol_gbp_info.session_close / symbol_euro_info.session_close, 1)
point_gbp = mt5.symbol_info(symbol_gbp).point
point_euro = mt5.symbol_info(symbol_euro).point
deviation = 1

# #Краткое описание каждого аргумента

# "action": mt5.TRADE_ACTION_DEAL,      # Совершить сделку прямо сейчас

# "symbol": symbol_gbp,                # Валюта сделки

# "volume": lot,                        # Объем который мы покупаем: У нас равен 1

# "type": mt5.ORDER_TYPE_SELL,          # Покупаем или продаём

# "deviation": deviation,               # Максимальное отклонение от заданной цены

# "magic": 123456,                      # Идентификатор эксперта. 
#                                       #Позволяет организовать аналитическую обработку торговых ордеров.    

# "comment": "Какой либо комментарий",  # Комменатрий к запросу

# "type_time": mt5.ORDER_TIME_GTC,      # Тип ордера по истечению: У нас Ордер будет находиться 
#                                       # в очереди до тех пор, пока не будет снят

# "type_filling": mt5.ORDER_FILLING_FOK,# Не будет продажи, если нет необходимого обьема 
    
    
request_gbp_sell_open = {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_gbp,
"volume": lotgbp, 
"type": mt5.ORDER_TYPE_SELL,
"deviation": deviation,
"magic": 123456,
"comment": "Открыли позицию, купили евро",
"type_time": mt5.ORDER_TIME_GTC,
"type_filling": mt5.ORDER_FILLING_FOK,
    

}
request_euro_buy_open = {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_euro,
"volume": loteuro,
"type": mt5.ORDER_TYPE_BUY,
"deviation": deviation,
"magic": 123456,
"comment": "Открыли позицию, продали фунт",
"type_time": mt5.ORDER_TIME_GTC,
"type_filling": mt5.ORDER_FILLING_FOK,      
}

request_gbp_buy_open = {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_gbp,
"volume": lotgbp,
"type": mt5.ORDER_TYPE_BUY, 
"deviation": deviation,
"magic": 123456,
"comment": "Открыли позицию, купили евро",
"type_time": mt5.ORDER_TIME_GTC,
"type_filling": mt5.ORDER_FILLING_FOK,  
}

request_euro_sell_open = {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_euro,
"volume": loteuro,
"type": mt5.ORDER_TYPE_SELL, 
"deviation": deviation,
"magic": 123456,
"comment": "Открыли позицию, продали фунт",
"type_time": mt5.ORDER_TIME_GTC,
"type_filling": mt5.ORDER_FILLING_FOK,      
}

# В запросах для закрытия позиции есть ещё один аргумент

#"position": order_gbp_sell.order,                     # Указывает какую позицию мы закрывем


# Чтобы запросы закрытия позции знали, что закрыват откроем предыдущие позции
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
order_gbp_buy_open = mt5.order_send(request_gbp_buy_open)
order_gbp_sell_open = mt5.order_send(request_gbp_sell_open)
order_euro_buy_open = mt5.order_send(request_euro_buy_open)
order_euro_sell_open = mt5.order_send(request_euro_sell_open)
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

request_gbp_buy_close = { 
"action": mt5.TRADE_ACTION_DEAL, 
"symbol": symbol_gbp, 
"volume": lotgbp, 
"type": mt5.ORDER_TYPE_BUY,
"position": order_gbp_sell_open.order,
"deviation": deviation, 
"magic": 123456, 
"comment": "купили евро", 
"type_time": mt5.ORDER_TIME_GTC, 
"type_filling": mt5.ORDER_FILLING_FOK, 
}
request_euro_sell_close = {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_euro,
"volume": loteuro,
"type": mt5.ORDER_TYPE_SELL,
"position": order_euro_buy_open.order,
"deviation": deviation,
"magic": 123456,
"comment": "продажа фунта",
"type_time": mt5.ORDER_TIME_GTC,
"type_filling": mt5.ORDER_FILLING_FOK,   
}
request_gbp_sell_close = { 
"action": mt5.TRADE_ACTION_DEAL, 
"symbol": symbol_gbp, 
"volume": lotgbp, 
"type": mt5.ORDER_TYPE_SELL,
"position": order_gbp_buy_open.order,
"deviation": deviation, 
"magic": 123456, 
"comment": "купили евро", 
"type_time": mt5.ORDER_TIME_GTC, 
"type_filling": mt5.ORDER_FILLING_FOK, 
}
request_euro_buy_close = {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_euro,
"volume": loteuro,
"type": mt5.ORDER_TYPE_BUY,
"position": order_euro_sell_open.order,
"deviation": deviation,
"magic": 123456,
"comment": "продажа фунта",
"type_time": mt5.ORDER_TIME_GTC,
"type_filling": mt5.ORDER_FILLING_FOK,   
}

# Функции для открытия и закрытия позиции

In [26]:
def request_open_up_pos(): 
    result = mt5.order_send(request_gbp_sell_open) 
    result2 = mt5.order_send(request_euro_buy_open)
    print('Открыл сверху')

In [27]:
def request_open_down_pos(): 
    result =mt5.order_send(request_gbp_buy_open) 
    result2 = mt5.order_send(request_euro_sell_open)
    print('Открыл снизу')

In [28]:
def request_close_up_pos():
    result = mt5.order_send(request_gbp_buy_close) 
    result2 = mt5.order_send(request_euro_sell_close)
    print('Закрыл сверху')

In [29]:
def request_close_down_pos():
    result = mt5.order_send(request_gbp_sell_close) 
    result2 = mt5.order_send(request_euro_buy_close)
    print('Закрыл снизу')

# Бесконечный цикл 1:
## Отрывает позицию в большом объеме и закрывает её как достигнет условие(Условие: тики раз в 10 мин за последний год)

In [31]:
#Достаём значения для открытия и закрытия позиции
values = calculation(take_rates()[0]['close'], take_rates()[1]['close'], loteuro)
Open_Up = values[0]
Open_Down = values[1]
Close = values[2]
#Некоторые переменные для подсчитывания открытых позиции и прибыли
Pos_Up = False
Pos_Down = False
quantity_open = 0
quantity_close = 0
balance_old = mt5.account_info().balance
while(True):
    if datetime.now().minute % 15 == 0 and datetime.now().second == 0:
        file_obj = open('QuantityOpenClose.txt', 'a')
        Quant_op = 'Quantity open: ' + str(quantity_open)+'\n'
        Quant_cl = 'Quantity close: ' + str(quantity_close)+'\n'
        Money = str(mt5.account_info().balance - balance_old) + '\n'
        balance_old = mt5.account_info().balance
        file_obj.write(Quant_op)
        file_obj.write(Quant_cl)
        file_obj.write(Money)
        file_obj.close()
        values = calculation(take_rates()[0]['close'], take_rates()[1]['close'], loteuro)
        Open_Up = values[0]
        Open_Down = values[1]
        Close = values[2]
        print(Open_Up, Open_Down, Close)
        time.sleep(1)
    last_tick_EURUSD = mt5.copy_rates_from("EURUSD", mt5.TIMEFRAME_M1, datetime.now() - timedelta(minutes=1), 1)['close'][0]
    last_tick_GBPUSD = mt5.copy_rates_from("GBPUSD", mt5.TIMEFRAME_M1, datetime.now() - timedelta(minutes=1), 1)['close'][0]
    last_spread = last_tick_GBPUSD - last_tick_EURUSD * loteuro
    if last_spread > Open_Up and Pos_Up == False:
        Pos_Up = True
        quantity_open += 1
        request_open_up_pos()        
    if last_spread < Open_Down and Pos_Down == False:
        Pos_Down = True
        quantity_open += 1
        request_open_down_pos()
        
    if last_spread <= Close and Pos_Up == True:
        Pos_Up = False
        quantity_close += 1
        request_close_up_pos()
        
    if last_spread >= Close and Pos_Down == True:
        Pos_Down = False
        quantity_close += 1
        request_close_down_pos()

Открыл снизу


KeyboardInterrupt: 

In [108]:
mt5.shutdown()

True

#  Далее черновой код не использующийся в основной программе